In [12]:
from bokeh.plotting import figure, save
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper
import geopandas as gpd
from bokeh.palettes import Viridis6 as palette
import geopandas as gpd
import pysal as ps
import pandas as pd
import numpy as np
import math



In [18]:
grid_fp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/zips/tl_2015_us_zcta510.shp"
network_fp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Transit Routes/Transit_Routes_for_King_County_Metro__transitroute_line.shp"
stops =pd.read_csv('C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Data/stops-zip.csv')

In [19]:
stops.head()

,Unnamed: 0,X,stop_id,stop_name,stop_lat,stop_lon,zone_id,stop_timezone,census_tract_id,zip
0,1,1,1000,Pine St & 9th Ave,47.613415,-122.332138,21,America/Los_Angeles,82.0,98101.0
1,2,2,10000,NE 55th St & 43rd Ave NE,47.668575,-122.283653,1,America/Los_Angeles,42.0,98105.0
2,3,3,10005,40th Ave NE & NE 51st St,47.665886,-122.284897,1,America/Los_Angeles,42.0,98105.0
3,4,4,10010,NE 55th St & 39th Ave NE,47.668579,-122.285667,1,America/Los_Angeles,42.0,98105.0
4,5,5,10020,NE 55th St & 37th Ave NE,47.668579,-122.288300,1,America/Los_Angeles,42.0,98105.0


In [14]:
grid = gpd.read_file(grid_fp)
network = gpd.read_file(network_fp)
grid = grid.loc[grid['ZCTA5CE10'].astype(float) >= 98001]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) <= 98198]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) != 98110]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) != 98068]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) != 98045]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) != 98022]
grid = grid.loc[grid['ZCTA5CE10'].astype(float) != 98051]


network = network.loc[network['ROUTE_NUM'].astype(str) != 'None']
network = network.loc[network['ROUTE_NUM'].astype(str) != '???']

In [15]:
def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]
    
def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )


grid['x'] = grid.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
grid['y'] = grid.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


In [16]:
network_source = GeoJSONDataSource(geojson=network.to_json())
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)

In [ ]:
#breaks = [x for x in range(1, 11, 2)]

# Initialize the classifier and apply it
#classifier = ps.User_Defined.make(bins=breaks)
#pt_classif = grid[['hh_income_detailed']].apply(classifier)

# Rename the classified column
#pt_classif.columns = ['income2']

# Join it back to the grid layer
#grid = grid.join(pt_classif)

In [22]:
from bokeh.palettes import RdYlGn10 as palette
from bokeh.models import LogColorMapper
from bokeh.plotting import figure, curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS 
from bokeh.plotting import figure, output_file, show 
from bokeh.events import Tap 

# Create the color mapper
TOOLS = "pan,wheel_zoom,reset,hover,poly_select,box_select,tap"

color_mapper = LogColorMapper(palette=palette)

p = figure(title="PSRC Income",tools=TOOLS)

# Plot grid
grid2 = p.patches('x', 'y', source=gsource,
         fill_color='white',
         fill_alpha=0.10, line_color="black", line_width=1, legend="income2")


#p.legend.location = "top_right"
#p.legend.orientation = "vertical"

r=p.multi_line('xs', 'ys', source=network_source, color="red", line_width=0.5)

ghover = HoverTool(renderers=[r])
ghover.tooltips=[("Bus Number", "@ROUTE_NUM")]

p.circle(x='stop_lon', y='stop_lat', size=1, color="green",source=stops)

def callback(attr, old, new):

    # The index of the selected glyph is : new['1d']['indices'][0]
    patch_name =  gsource.data['ZCTA5CE10'][new['1d']['indices'][0]]
    print("TapTool callback executed on Patch {}".format(patch_name))

grid2.data_source.on_change('selected',callback)

curdoc().add_root(column(p))

p.add_tools(ghover)

#outfp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Docs/map.html"
#save(p, outfp)

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name:  [renderer: GlyphRenderer(id='f69f3f53-a5ad-4bbd-9f47-28f628ead95c', ...)]
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='23a04cf7-3374-41f5-9d2c-cf7ef5300ba4', ...)
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name:  [renderer: GlyphRenderer(id='f69f3f53-a5ad-4bbd-9f47-28f628ead95c', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name:  [renderer: GlyphRenderer(id='80268d84-9b70-4988-ae6e-100c1504ae33', ...)]
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='817e5a72-a546-4242-85ec-22e54dee473e', ...)
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name:  [renderer: GlyphRenderer(id='80268d84-9b70-4988-ae6e-100c1504ae33', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name:  [renderer: GlyphRenderer(id='21baa38c-bd82-4cf4-af67-c9d6fa1e9